In [3]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import re 
from shutil import copyfile

In [70]:
parent = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_raw"
dirs = os.listdir(parent)

meta_data = []
for folder in dirs:
    xml_loc = glob.glob(parent + "/" + folder + "/*.xml")[0]
    tree = ET.parse(xml_loc)
    directory = [elem.text for elem in tree.iter() if "MASK_FILENAME" in elem.tag][0].split("/")[1]
    tile_id = directory.split("_")[1]
    filepath_partial = parent + "/" + directory + "/IMG_DATA" + "/R10m"
    filepath = glob.glob(filepath_partial + "/*.*")[0]
    filename = filepath.split("/")[-1]
    cloud_cover,no_data,unclassified = [elem.text for elem in tree.iter() if "CLOUDY_PIXEL_PERCENTAGE" in elem.tag 
             or "NODATA_PIXEL_PERCENTAGE" in elem.tag or "UNCLASSIFIED_PERCENTAGE" in elem.tag]
    meta_data.append([directory,tile_id,cloud_cover,no_data,unclassified,filename,filepath])
df = pd.DataFrame(meta_data,columns=["Directory","Tile_Id","Cloud Cover","No Data Percentage","Unclassified Percentage","Filename","Filepath"])




In [80]:
#export df to csv

In [83]:
#read in dataframe

df2 = pd.read_csv("./data/Dataframe/tile_list.csv").sort_values(by=["Tile_Id","Unclassified Percentage"],ignore_index=True)

In [84]:
df2["Tile_Id"].value_counts()

T33NYC    10
T34MBC    10
T34NFG    10
T34MBB    10
T33NVD    10
T33NVA    10
T34NDF    10
T34NEF    10
T34NDH    10
T34NFH    10
T33NVC    10
T33NWA    10
T33NZD    10
T33NWB    10
T34MCB    10
T33NXB    10
T33NUE    10
T33NYD    10
T33NUC     9
T33NUA     8
T33NUB     7
T33NUD     3
T33NTB     2
T33NTC     1
Name: Tile_Id, dtype: int64

In [85]:
base_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered/"
layer = 1
df = df2
for index,row in df.iterrows(): 
    destination_dir = base_dir + str(layer)
    print("destination_dir:",destination_dir)
    output_file = destination_dir + "/" + row["Filename"]
    print("output_file:",output_file)
    
    # Check if directory exists
    if not os.path.isdir(destination_dir):
        os.mkdir(destination_dir)
        
    # Copy file to existing or new directory
    copyfile(row["Filepath"],output_file)
    
    # Check if Tile_Id already exists in the directory - only necessary up until the last tile
    if len(df) > index + 1:
        if df.loc[index,"Tile_Id"] == df.loc[index + 1,"Tile_Id"]:
            layer += 1
        else:
            layer = 1   

destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered/1
output_file: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered/1/T33NTB_20191110T092119_TCI_10m.jp2
destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered/2
output_file: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered/2/T33NTB_20200806T092029_TCI_10m.jp2
destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered/1
output_file: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered/1/T33NTC_20200124T092251_TCI_10m.jp2
destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered/1
outp

In [1]:
# for row in range(len(df5)):
#     layer = 1
#     dest_folder = f'.../layer_{layel}'
#     # Copy TCI to dest_folder
#     if not os.exists(dest_folder):
#         os.mkdir(dest_folder)

#     if row != len(df5)-1:
#         if df5.loc[row, 'tileid_new'] == df5.loc[row+1, 'tileid_new']:
#             layer += 1
#         else:
#             layer = 1

# For Finding Corresponding SCL File for each tile from SAFE_true

In [28]:
source_dir_SAFE_true = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_raw/"
source_dir_SAFE_false = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_false_raw/"

xml_source = glob.glob(source_dir_SAFE_true + "*/*.xml")[0]

print(os.listdir(source_dir_SAFE_false)[0],os.listdir(source_dir_SAFE_true)[0])

SAFE_false_dir = os.listdir(source_dir_SAFE_false)[0]
SAFE_true_dir = os.listdir(source_dir_SAFE_true)

34MBC,2020-08-25,0 L2A_T34MBC_A027031_20200825T092011


In [25]:
tile_id = SAFE_false_dir.split(",")[0]
tile_date = SAFE_false_dir.split(",")[1]

In [32]:
SAFE_true_dir[0].split("_")[1]

'T34MBC'